# API Entry Points — Report

This notebook visualizes the CSV outputs generated by the **API_Entry_Points** block.

## What this notebook shows
- **Main_Classes**: distribution of `visibility`, `static` vs. `non-static`, top classes by frequency, signatures.
- **Spring_Controller**: controllers per stereotype (Controller / RestController), top controllers.
- **Spring_Endpoints**: sunburst for `controller → httpMethod → endpoint`, HTTP method distribution, endpoints per controller.

> If a CSV is missing or empty, the cell prints an info message and skips the chart.


In [1]:
# Setup: imports, paths, helpers
# - Keep plotting logic intact; only path resolution is simplified and standardized.
# - CSVs live under ../reports/csv-reports/<CATEGORY>/<file>.csv relative to this notebook folder.
# - Minimal console output; only show information if a CSV is missing/empty.

import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import textwrap

# Base folders and category for this notebook
CATEGORY = "API_Entry_Points"
CSV_BASE = Path("../reports/csv-reports").resolve()
API_DIR = CSV_BASE / CATEGORY

# Avoid downcasting warnings when masking NAs
pd.set_option('future.no_silent_downcasting', True)

def read_csv_safe(path: Path) -> pd.DataFrame:
    """Read a CSV if present; otherwise return an empty DataFrame.
    Prints a minimal info message when missing or unreadable."""
    path = Path(path)
    if not path.exists():
        print(f"[info] Missing CSV: {path}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(path)
        df.columns = [str(c).strip() for c in df.columns]
        return df
    except Exception as e:
        print(f"[warn] Failed to read {path}: {e}")
        return pd.DataFrame()

def show_head(df: pd.DataFrame, n: int = 8):
    """Display a quick head for ad‑hoc inspection; silent if empty."""
    if df.empty:
        print("[info] DataFrame is empty.")
    else:
        display(df.head(n))

def shorten_label(s: str, maxlen: int = 40) -> str:
    s = str(s)
    return (s[:maxlen-1] + "…") if len(s) > maxlen else s

def wrap_label_html(s: str, width: int = 28) -> str:
    return "<br>".join(textwrap.wrap(str(s), width=width))

def fillna_safe(series, value):
    """Mask NA values with a given literal without triggering downcast warnings."""
    s = series.copy()
    s = s.mask(s.isna(), value)
    return s


## 1) Main Classes

In [ ]:
# Charts for Main_Classes
# Using shared functions from interface/charts/entry_points_charts.py

# Import chart creation functions
import sys
from pathlib import Path
sys.path.append(str(Path("../interface").resolve()))
from charts.entry_points_charts import (
    create_visibility_chart,
    create_static_chart,
    create_top_classes_chart,
    create_signature_chart
)

# Expected columns: mainClass, isStatic, visibility, signature
path = API_DIR / "Main_Classes.csv"
df_main = read_csv_safe(path)
show_head(df_main)

if not df_main.empty:
    # Normalize column names to lowercase for easier access
    cols = {c.lower(): c for c in df_main.columns}
    c_main = cols.get("mainclass")
    c_static = cols.get("isstatic")
    c_vis = cols.get("visibility")
    c_sig = cols.get("signature")
    
    # 1A) Visibility distribution
    fig = create_visibility_chart(df_main, c_vis)
    if fig:
        fig.show()
    else:
        print("[info] Column 'visibility' not found — skipping visibility chart.")
    
    # 1B) Static vs non-static
    fig = create_static_chart(df_main, c_static)
    if fig:
        fig.show()
    else:
        print("[info] Column 'isStatic' not found — skipping static/non-static chart.")
    
    # 1C) Top classes exposing main()
    fig = create_top_classes_chart(df_main, c_main)
    if fig:
        fig.show()
    else:
        print("[info] Column 'mainClass' not found — skipping top classes chart.")
    
    # 1D) Signature variants
    fig = create_signature_chart(df_main, c_sig)
    if fig:
        fig.show()
    else:
        print("[info] Column 'signature' not found — skipping signature chart.")
else:
    print("[info] Main_Classes.csv missing or empty.")

## 2) Spring Controllers

In [ ]:
# Charts for Spring_Controller
# Using shared functions from interface/charts/entry_points_charts.py

# Import chart creation functions (add to existing imports if needed)
if 'create_controller_stereotypes_chart' not in dir():
    from charts.entry_points_charts import (
        create_controller_stereotypes_chart,
        create_top_controllers_chart
    )

# Expected columns: ControllerClassFqn, Package (stereotype fqns)
path = API_DIR / "Spring_Controller.csv"
df_ctrl = read_csv_safe(path)
show_head(df_ctrl)

if not df_ctrl.empty:
    # Normalize column names to lowercase for easier access
    cols = {c.lower(): c for c in df_ctrl.columns}
    c_cls = cols.get("controllerclassfqn") or cols.get("controller") or cols.get("classname")
    c_pkg = cols.get("package")
    
    # 2A) Stereotype distribution
    fig = create_controller_stereotypes_chart(df_ctrl, c_pkg)
    if fig:
        fig.show()
    else:
        print("[info] Column 'Package' not found — skipping stereotype pie.")
    
    # 2B) Top Controllers
    fig = create_top_controllers_chart(df_ctrl, c_cls)
    if fig:
        fig.show()
    else:
        print("[info] Column 'ControllerClassFqn' not found — skipping top controllers chart.")
else:
    print("[info] Spring_Controller.csv missing or empty.")

## 3) Spring Endpoints

In [ ]:
# Charts for Spring_Endpoints
# Using shared functions from interface/charts/entry_points_charts.py

# Import chart creation functions (add to existing imports if needed)
if 'create_endpoints_sunburst_chart' not in dir():
    from charts.entry_points_charts import (
        create_endpoints_sunburst_chart,
        create_http_method_distribution_chart,
        create_endpoints_per_controller_chart
    )

# Expected columns: controller, method, httpMethod, completeEndpoint
path = API_DIR / "Spring_Endpoints.csv"
df_ep = read_csv_safe(path)
show_head(df_ep)

if not df_ep.empty:
    # Normalize column names to lowercase for easier access
    cols = {c.lower(): c for c in df_ep.columns}
    c_ctrl = cols.get("controller")
    c_meth = cols.get("method")
    c_http = cols.get("httpmethod") or cols.get("httprequestmethod") or cols.get("methodtype")
    c_path = cols.get("completeendpoint") or cols.get("path") or cols.get("endpoint")
    
    # Ensure string types
    if c_ctrl: df_ep[c_ctrl] = df_ep[c_ctrl].astype(str)
    if c_meth: df_ep[c_meth] = df_ep[c_meth].astype(str)
    if c_http: df_ep[c_http] = df_ep[c_http].astype(str)
    if c_path: df_ep[c_path] = df_ep[c_path].astype(str)
    
    # 3A) Sunburst large
    fig = create_endpoints_sunburst_chart(df_ep, c_ctrl, c_http, c_path)
    if fig:
        fig.show()
    else:
        print("[info] Missing one of [controller, httpMethod, completeEndpoint] — skipping sunburst.")
    
    # 3B) HTTP method distribution
    fig = create_http_method_distribution_chart(df_ep, c_http)
    if fig:
        fig.show()
    else:
        print("[info] Column 'httpMethod' not found — skipping HTTP method distribution.")
    
    # 3C) Endpoints per controller (Top 30)
    fig = create_endpoints_per_controller_chart(df_ep, c_ctrl, c_path)
    if fig:
        fig.show()
    else:
        print("[info] Missing controller/path — skipping endpoints per controller.")
else:
    print("[info] Spring_Endpoints.csv missing or empty.")